# Replicating ProPublica's COMPAS Audit

## Why COMPAS?


Propublica started the COMPAS Debate with the article [Machine Bias](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencin).  With their article, they also released details of their methodology and their [data and code](https://github.com/propublica/compas-analysis).  This presents a real data set that can be used for research on how data is used in a criminal justice setting without researchers having to perform their own requests for information, so it has been used and reused a lot of times.

In [3]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from sklearn.metrics import roc_curve
import warnings
warnings.filterwarnings('ignore')

In [4]:
propublica_data_url = 'https://github.com/propublica/compas-analysis/raw/master/compas-scores-two-years.csv'
df_pp = pd.read_csv(propublica_data_url,
                 header=0).set_index('id')

In [6]:
df_pp.head()

,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
id,,,,,,,,,,,,,,,,,,,,,
1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,0,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,0,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,0,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


In [9]:
df_pp.tail()

,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
id,,,,,,,,,,,,,,,,,,,,,
10996,steven butler,steven,butler,2013-11-23,Male,1992-07-17,23,Less than 25,African-American,0,...,5,Medium,2013-11-23,2013-11-22,2013-11-24,0,1,860,0,0
10997,malcolm simmons,malcolm,simmons,2014-02-01,Male,1993-03-25,23,Less than 25,African-American,0,...,5,Medium,2014-02-01,2014-01-31,2014-02-02,0,1,790,0,0
10999,winston gregory,winston,gregory,2014-01-14,Male,1958-10-01,57,Greater than 45,Other,0,...,1,Low,2014-01-14,2014-01-13,2014-01-14,0,0,808,0,0
11000,farrah jean,farrah,jean,2014-03-09,Female,1982-11-17,33,25 - 45,African-American,0,...,2,Low,2014-03-09,2014-03-08,2014-03-09,3,0,754,0,0
11001,florencia sanmartin,florencia,sanmartin,2014-06-30,Female,1992-12-18,23,Less than 25,Hispanic,0,...,4,Low,2014-06-30,2015-03-15,2015-03-15,2,0,258,0,1


In [10]:
df_pp.bottom()

AttributeError: 'DataFrame' object has no attribute 'bottom'

In [11]:
df_pp.tail()

,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
id,,,,,,,,,,,,,,,,,,,,,
10996,steven butler,steven,butler,2013-11-23,Male,1992-07-17,23,Less than 25,African-American,0,...,5,Medium,2013-11-23,2013-11-22,2013-11-24,0,1,860,0,0
10997,malcolm simmons,malcolm,simmons,2014-02-01,Male,1993-03-25,23,Less than 25,African-American,0,...,5,Medium,2014-02-01,2014-01-31,2014-02-02,0,1,790,0,0
10999,winston gregory,winston,gregory,2014-01-14,Male,1958-10-01,57,Greater than 45,Other,0,...,1,Low,2014-01-14,2014-01-13,2014-01-14,0,0,808,0,0
11000,farrah jean,farrah,jean,2014-03-09,Female,1982-11-17,33,25 - 45,African-American,0,...,2,Low,2014-03-09,2014-03-08,2014-03-09,3,0,754,0,0
11001,florencia sanmartin,florencia,sanmartin,2014-06-30,Female,1992-12-18,23,Less than 25,Hispanic,0,...,4,Low,2014-06-30,2015-03-15,2015-03-15,2,0,258,0,1


In [13]:
df_pp.shape

(7214, 52)

## Data Cleaning

In [21]:
clean_data_url = 'https://raw.githubusercontent.com/ml4sts/outreach-compas/main/data/compas_c.csv'
df = pd.read_csv(clean_data_url,
                 header=0).set_index('id')

In [22]:
df_pp.tail()

,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
id,,,,,,,,,,,,,,,,,,,,,
10996,steven butler,steven,butler,2013-11-23,Male,1992-07-17,23,Less than 25,African-American,0,...,5,Medium,2013-11-23,2013-11-22,2013-11-24,0,1,860,0,0
10997,malcolm simmons,malcolm,simmons,2014-02-01,Male,1993-03-25,23,Less than 25,African-American,0,...,5,Medium,2014-02-01,2014-01-31,2014-02-02,0,1,790,0,0
10999,winston gregory,winston,gregory,2014-01-14,Male,1958-10-01,57,Greater than 45,Other,0,...,1,Low,2014-01-14,2014-01-13,2014-01-14,0,0,808,0,0
11000,farrah jean,farrah,jean,2014-03-09,Female,1982-11-17,33,25 - 45,African-American,0,...,2,Low,2014-03-09,2014-03-08,2014-03-09,3,0,754,0,0
11001,florencia sanmartin,florencia,sanmartin,2014-06-30,Female,1992-12-18,23,Less than 25,Hispanic,0,...,4,Low,2014-06-30,2015-03-15,2015-03-15,2,0,258,0,1


In [24]:
df_pp.shape

(7214, 52)

In [25]:
df.tail

<bound method NDFrame.tail of        age c_charge_degree              race       age_cat score_text     sex  \
id                                                                              
3       34               F  African-American       25 - 45        Low    Male   
4       24               F  African-American  Less than 25        Low    Male   
8       41               F         Caucasian       25 - 45     Medium    Male   
10      39               M         Caucasian       25 - 45        Low  Female   
14      27               F         Caucasian       25 - 45        Low    Male   
...    ...             ...               ...           ...        ...     ...   
10994   30               M  African-American       25 - 45        Low    Male   
10995   20               F  African-American  Less than 25       High    Male   
10996   23               F  African-American  Less than 25     Medium    Male   
10997   23               F  African-American  Less than 25        Low    Male  

In [26]:
## Data Exploration


Next we provide a few ways to look at the relationships between the attributes in the dataset. Here is an explanation of these values:


* `age`: defendant's age
* `c_charge_degree`: degree charged (Misdemeanor of Felony)
* `race`: defendant's race
* `age_cat`: defendant's age quantized in "less than 25", "25-45", or "over 45"
* `score_text`: COMPAS score: 'low'(1 to 5), 'medium' (5 to 7), and 'high' (8 to 10).
* `sex`: defendant's gender
* `priors_count`: number of prior charges
* `days_b_screening_arrest`: number of days between charge date and arrest where defendant was screened for compas score
* `decile_score`: COMPAS score from 1 to 10 (low risk to high risk)
* `is_recid`: if the defendant recidivized
* `two_year_recid`: if the defendant within two years
* `c_jail_in`: date defendant was imprisoned
* `c_jail_out`: date defendant was released from jail
* `length_of_stay`: length of jail stay


SyntaxError: invalid syntax (<ipython-input-26-8d27197781a0>, line 4)

## Data Exploration


Next we provide a few ways to look at the relationships between the attributes in the dataset. Here is an explanation of these values:


* `age`: defendant's age
* `c_charge_degree`: degree charged (Misdemeanor of Felony)
* `race`: defendant's race
* `age_cat`: defendant's age quantized in "less than 25", "25-45", or "over 45"
* `score_text`: COMPAS score: 'low'(1 to 5), 'medium' (5 to 7), and 'high' (8 to 10).
* `sex`: defendant's gender
* `priors_count`: number of prior charges
* `days_b_screening_arrest`: number of days between charge date and arrest where defendant was screened for compas score
* `decile_score`: COMPAS score from 1 to 10 (low risk to high risk)
* `is_recid`: if the defendant recidivized
* `two_year_recid`: if the defendant within two years
* `c_jail_in`: date defendant was imprisoned
* `c_jail_out`: date defendant was released from jail
* `length_of_stay`: length of jail stay


In [27]:
df['race'].value_counts()

African-American    3175
Caucasian           2103
Name: race, dtype: int64